# Integrate Agenta with Google ADK

This notebook demonstrates how to connect **Agenta** with **Google ADK** for comprehensive observability and debugging of your AI agent applications.

> **What is Agenta?** [Agenta](https://agenta.ai) is an open-source LLMOps platform designed to streamline the deployment, management, and scaling of large language models. It offers comprehensive observability, testing, and deployment capabilities for AI applications.

> **What is Google ADK?** [Google ADK](https://github.com/google/adk) (Agent Development Kit) is Google's framework for building AI agents powered by Gemini models. It provides tools for creating agents with custom capabilities, structured workflows, and seamless integration with Google's AI ecosystem.

## Implementation Guide

Follow this tutorial to set up Google ADK with Agenta’s observability platform and enable end-to-end tracing for your agent workflows.

### Step 1: Install Required Dependencies

Install the necessary Python packages for this integration:

In [ ]:
!pip install agenta google-adk openinference-instrumentation-google-adk nest_asyncio

**Package Descriptions:**
- `agenta`: Core SDK for Agenta's prompt engineering and observability platform
- `google-adk`: Google's Agent Development Kit for building AI agents powered by Gemini models
- `openinference-instrumentation-google-adk`: Automatic instrumentation library for Google ADK operations

### Step 2: Setup and Configuration

Configure your environment and initialize the Agenta SDK:

In [ ]:
import os
import nest_asyncio

import agenta as ag
from openinference.instrumentation.google_adk import GoogleADKInstrumentor

# Enable nested event loops inside Jupyter
nest_asyncio.apply()

# Load configuration from environment
os.environ["AGENTA_API_KEY"] = "YOUR AGENTA API KEY"
os.environ["AGENTA_HOST"] = "https://cloud.agenta.ai"
os.environ["GOOGLE_API_KEY"] = "YOUR GOOGLE API KEY"  # Required for Google ADK / Gemini

# Initialize Agenta (uses AGENTA_* env vars)
ag.init()

**What does `ag.init()` do?**
This function initializes the Agenta SDK and sets up the necessary configuration for observability. It establishes connection to the Agenta platform, configures tracing and logging settings, and prepares the instrumentation context for your application.

### Step 3: Enable Google ADK Monitoring

Initialize the OpenInference Google ADK instrumentation to automatically capture agent operations:

In [ ]:
# Enable Google ADK instrumentation
GoogleADKInstrumentor().instrument()

### Step 4: Build Your Instrumented Google ADK Application

Here's a complete example showcasing a weather agent with Agenta instrumentation:

#### Setup Weather Data and Agent

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

APP_NAME = "weather_app"
USER_ID = "demo_user"


def get_weather(city: str) -> dict:
    """Toy tool used to generate spans in our traces."""
    normalized = city.strip().lower()
    if normalized == "new york":
        return {
            "status": "success",
            "report": "The weather in New York is sunny with a temperature of 25°C.",
        }
    if normalized == "london":
        return {
            "status": "success",
            "report": "The weather in London is cloudy with a temperature of 18°C.",
        }
    else:
        return {
            "status": "error",
            "error_message": f"Weather information for '{city}' is not available.",
        }


weather_agent = Agent(
    name="weather_agent",
    model="gemini-2.0-flash-exp",
    description="Agent that answers simple weather questions using a tool.",
    instruction="You must use the tools to answer the user's question.",
    tools=[get_weather],
)

session_service = InMemorySessionService()
weather_runner = Runner(
    agent=weather_agent, app_name=APP_NAME, session_service=session_service
)

#### Create Instrumented Handler Function

In [ ]:
import agenta as ag


@ag.instrument(spankind="workflow")
async def ask_weather(question: str, user_id: str = "demo_user") -> str:
    """
    Run a single weather question through the Google ADK agent.
    This appears as a top-level span inside Agenta observability.
    """

    # Create a session for this user
    session = await session_service.create_session(
        app_name=APP_NAME,
        user_id=user_id,
    )

    content = types.Content(
        role="user",
        parts=[types.Part.from_text(text=question)],
    )

    try:
        events = weather_runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=content,
        )

        final_text = ""
        async for event in events:
            if event.is_final_response():
                final_text = event.content.parts[0].text.strip()

        return final_text

    except Exception as exc:
        # Basic handling for Gemini quota / resource exhaustion
        msg = str(exc).lower()
        if "exhausted" in msg:
            return "The model is temporarily exhausted or over quota. (Check you 'google gemini' subscription) "
        # Re-raise all other errors so you still see real issues
        raise

#### Test the Weather Agent

In [ ]:
# Example usage
async def main():
    response = await ask_weather("What is the weather in New York?")
    print("Response:", response)


# Run the example
await main()

### Step 5: Understanding the @ag.instrument() Decorator

The `@ag.instrument()` decorator automatically captures all input and output data from your function, enabling comprehensive observability without manual instrumentation.

**Span Type Configuration:**
Use the `spankind` parameter to categorize operations in Agenta WebUI. Available options:

- `agent` - Autonomous agent behaviors
- `chain` - Sequential processing workflows
- `workflow` - Complete application processes (default)
- `tool` - Utility and helper functions
- `embedding` - Vector embedding operations
- `query` - Search and retrieval tasks
- `completion` - Text generation operations
- `chat` - Conversational interfaces
- `rerank` - Result ordering operations

**Standard Behavior:**
By default, when `spankind` is not specified, the operation becomes a root-level span, categorized as a `workflow` in Agenta.

In [ ]:
# Example with custom span classification:
import agenta as ag


@ag.instrument(spankind="agent")
def specialized_agent_function(input_data: str):
    # Agent-specific logic implementation (placeholder)
    return input_data.upper()

### Step 6: View Traces in Agenta

After running your instrumented function, you can inspect all execution traces inside the Agenta dashboard.  
The recorded observability data typically includes:

- Top-level workflow span for your `ask_weather()` call  
- Nested spans generated by Google ADK  
- Tool invocation spans (e.g., `get_weather`)  
- Model call information for Gemini requests  
- Input/output data captured by the instrumentation  
- Execution timing and performance metrics  

These traces help you:

- Understand how your Google ADK agent processes a request  
- Debug tool behaviors and model interactions  
- Monitor latency and performance  
- Verify that your agent is behaving as expected

You can access all traces under **Observability → Traces** in the Agenta UI.

## Next Steps

For more detailed information about Agenta's observability features and advanced configuration options, visit the [Agenta Observability SDK Documentation](/observability/observability-sdk).